In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 100.2 MB/s eta 0:00:00


# **Data**

# Covid Worry dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np

covidworry_train_csv_path = '/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/covidworry/covidworry_train.csv'
covidworry_dev_csv_path = '/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/covidworry/covidworry_test.csv'

covidworry_train = pd.read_csv(covidworry_train_csv_path)
covidworry_dev = pd.read_csv(covidworry_dev_csv_path)


In [5]:
print('covidworry_train shape:', covidworry_train.shape)
covidworry_train['essay'].head(10)

covidworry_train shape: (1685, 2)


0    I feel stressed because of self isolation and ...
1    At this moment, I am feeling incredibly bored ...
2    Im feeling a combination of anxiety and fear f...
3    Extremely anxious at the Unknown’s in the whol...
4    I feel worried for my friends and family and s...
5    I am worried about family and friends being af...
6    I am scared for my family and friends. I do no...
7    It's a slightly worrying situation especially ...
8    I am quite anxious at the moment as I have a p...
9    Very worried I might get it if go out for supp...
Name: essay, dtype: object

In [6]:
covidworry_train['emotion'].head(10)

0    3
1    0
2    3
3    3
4    1
5    0
6    0
7    1
8    3
9    4
Name: emotion, dtype: int64

In [7]:
X_train = covidworry_train['essay']
y_train = covidworry_train['emotion']

X_test = covidworry_dev['essay']
y_test = covidworry_dev['emotion']

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1685,)
(1685,)
(723,)
(723,)


In [9]:
# label: 'sadness': 0, 'neutral': 6, 'fear': 2, 'anger': 3, 'disgust': 4, 'surprise': 5, 'joy': 6
y_train.value_counts()

3    966
4    250
1    233
0    161
2     75
Name: emotion, dtype: int64

In [10]:
y_test.value_counts()

3    415
4    107
1    100
0     69
2     32
Name: emotion, dtype: int64

# **Model**


In [11]:
# Fit and transform X_train using Tfidf Vectorizer with default parameters
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
print('tfidf train shape:', X_train_tfidf.shape)
print('tfidf train type:', X_train_tfidf.dtype)

X_test_tfidf = vectorizer.transform(X_test).toarray()
print('tfidf test:', X_test_tfidf.shape)


tfidf train shape: (1685, 7142)
tfidf train type: float64
tfidf test: (723, 7142)


In [12]:
from sklearn.svm import LinearSVC
clf = LinearSVC().fit(X_train_tfidf, y_train)

from sklearn import metrics
predicted = clf.predict(X_test_tfidf)

acc = metrics.accuracy_score(y_test, predicted)
print('accuracy is: ', acc*100)

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predicted))
print('\n')
print(classification_report(y_test,predicted))

accuracy is:  56.984785615491006
[[  6   5   0  51   7]
 [  2  26   0  64   8]
 [  1   6   0  20   5]
 [ 10  21   0 368  16]
 [  1   8   0  86  12]]


              precision    recall  f1-score   support

           0       0.30      0.09      0.13        69
           1       0.39      0.26      0.31       100
           2       0.00      0.00      0.00        32
           3       0.62      0.89      0.73       415
           4       0.25      0.11      0.15       107

    accuracy                           0.57       723
   macro avg       0.31      0.27      0.27       723
weighted avg       0.48      0.57      0.50       723



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Augmentation using Embedding Word2Vec

In [13]:
! pip install nlpaug==1.1.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 KB 8.5 MB/s eta 0:00:00


In [14]:
from nlpaug.util.file.download import DownloadUtil
DownloadUtil.download_glove('glove.6B', '.')

In [15]:
# apply augmentation to the train data and save the results into a file
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action
import nlpaug.flow as naf

aug_emb = naf.Sequential([naf.Sometimes([
    naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="substitute")]),
    naf.Sometimes([naw.WordEmbsAug(
    model_type='glove', model_path='./glove.6B.200d.txt',
    action="insert"),
])])


In [16]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [17]:
from tqdm.notebook import tqdm
tqdm.pandas()

data = {'text': X_train, 'label': y_train}
base_train = pd.DataFrame.from_dict(data)
base_train.shape

(1685, 2)

In [18]:
rep = 5 # how many repetitions of the record


emb_train = base_train.copy()
emb_train['paraphrase'] = emb_train['text'].progress_apply(lambda x:aug_emb.augment(x, rep))
emb_train = emb_train.explode('paraphrase').reset_index(drop=True)

  0%|          | 0/1685 [00:00<?, ?it/s]

In [19]:
emb_train.head(20)

,text,label,paraphrase
0,I feel stressed because of self isolation and ...,3,I feel stressed because 2.257 and self isolati...
1,I feel stressed because of self isolation and ...,3,ej I feel stressed because of self isolation a...
2,I feel stressed because of self isolation and ...,3,I feel discussions because up less isolation a...
3,I feel stressed because of self isolation and ...,3,I feel stressed because from self isolation an...
4,I feel stressed because of self isolation and ...,3,I feel stressed magrino because of self isolat...
5,"At this moment, I am feeling incredibly bored ...",0,"At this clogged moment, I definitely feeling i..."
6,"At this moment, I am feeling incredibly bored ...",0,"At this moment, I am feeling incredibly bored ..."
7,"At this moment, I am feeling incredibly bored ...",0,"At this moment, I am feeling incredibly pretty..."
8,"At this moment, I am feeling incredibly bored ...",0,"hasemi At this moment, I am feeling incredibly..."
9,"At this moment, I am feeling incredibly bored ...",0,"At this moment, I am balranald feeling incredi..."


# Model with augmented train data (paraphrased data)

In [20]:
# Fit and transform X_train using Tfidf Vectorizer with default parameters
from sklearn.feature_extraction.text import TfidfVectorizer

# !! we should concatenate original train with paraphrased train
new_data = X_train.append(emb_train['paraphrase'])

vectorizer = TfidfVectorizer() #max_features=3000)
X_train_tfidf = vectorizer.fit_transform(new_data).toarray() #emb_train['paraphrase']).toarray()
print('tfidf train shape:', X_train_tfidf.shape)
print('tfidf train type:', X_train_tfidf.dtype)

X_test_tfidf = vectorizer.transform(X_test).toarray()
print('tfidf test:', X_test_tfidf.shape)


tfidf train shape: (10110, 59170)
tfidf train type: float64
tfidf test: (723, 59170)


In [21]:
from sklearn.svm import LinearSVC
# !! we should concatenate original labels with paraphrased text labels
new_labels = y_train.append(emb_train['label'])

clf = LinearSVC().fit(X_train_tfidf, new_labels) # emb_train['label'])

from sklearn import metrics
predicted = clf.predict(X_test_tfidf)

acc = metrics.accuracy_score(y_test, predicted)
print('accuracy is: ', acc*100)

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predicted))
print('\n')
print(classification_report(y_test,predicted))

accuracy is:  54.08022130013831
[[  9   5   0  43  12]
 [  2  34   0  55   9]
 [  3   5   0  19   5]
 [ 20  33   3 329  30]
 [  5   9   0  74  19]]


              precision    recall  f1-score   support

           0       0.23      0.13      0.17        69
           1       0.40      0.34      0.37       100
           2       0.00      0.00      0.00        32
           3       0.63      0.79      0.70       415
           4       0.25      0.18      0.21       107

    accuracy                           0.54       723
   macro avg       0.30      0.29      0.29       723
weighted avg       0.48      0.54      0.50       723



# Create CSV file from augmented dataset

In [22]:
!cd '/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/'
emb_train.to_csv('/content/drive/My Drive/Colab Notebooks/NLP Augmentation/Datasets/covidworry_emb_aug_5.csv', encoding='utf-8', index=False, sep=',')